In [1]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.notebook import tqdm

import MST

In [2]:
BATCH_SIZE = 32
NUM_WORKERS = 0

EPOCHS = 10

In [3]:
transform = np.array

train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.MNIST(
    root='datasets',
    train=False,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [4]:
class MyNet(MST.BasicModule):
    def __init__(self):
        super().__init__()

        self.net_blocks = MST.Sequential(

                MST.FullyConnectedLayer(28*28, 100),
                MST.Relu(),

                MST.Sequential(
                            MST.FullyConnectedLayer(100, 50),
                            MST.Relu(),
                        ),

                MST.Sequential(
                            MST.FullyConnectedLayer(50, 10),
                            MST.Relu()
                        ),
        )

    def forward(self, x):
        x = x.reshape(-1, 28*28)
        x = self.net_blocks(x)
        return x

    def backward(self, dOut):
        dN = self.net_blocks.backward(dOut)
        return dN

CELoss = MST.CrossEntropyLoss()
net = MyNet()

optimizer = MST.SGD(net, lr=0.001, momentum=0.9)
print(net)

MyNet:
└── net_blocks: 
	 └── FullyConnectedLayer: Trainable(True)
	 └── Relu: Trainable(False)
	 └── Sequential: 
		 └── FullyConnectedLayer: Trainable(True)
		 └── Relu: Trainable(False)

	 └── Sequential: 
		 └── FullyConnectedLayer: Trainable(True)
		 └── Relu: Trainable(False)


In [5]:
def train(net : MST.BasicModule, optimizer : MST.SGD, criterion : MST.BasicModule):
    running_loss = 0
    for images, labels in train_dataloader:
        labels = np.array(labels)
        
        output = net(images)

        loss = criterion(output, labels)
        net.backward(criterion.backward())
        optimizer.step()

        running_loss += loss
    train_loss = running_loss / len(train_dataloader)
    return train_loss


def valid(net : MST.BasicModule, criterion : MST.BasicModule):
    running_loss = 0
    correct_total = 0
    for images, labels in test_dataloader:
        labels = np.array(labels)

        output = net(images)

        loss = criterion(output, labels)
        running_loss += loss

        pred = np.argmax(output, axis=1, keepdims=True)
        correct_total += np.sum(pred[pred==labels.reshape(-1, 1)].astype(bool))
        
    precison = correct_total / len(test_dataloader.dataset)
    valid_loss = running_loss / len(test_dataloader)
    return valid_loss, precison

In [6]:
for epoch in (pbar := tqdm(range(EPOCHS))):
    train_loss = train(net, optimizer, CELoss)
    valid_loss, prec = valid(net, CELoss)

    print(f"[{epoch}] train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")
    pbar.set_description(f"train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

[0] train/valid loss: 2.3583/2.3025 prec: 0.1028
[1] train/valid loss: 2.3025/2.3026 prec: 0.1028
[2] train/valid loss: 2.3025/2.3025 prec: 0.1028
[3] train/valid loss: 2.3025/2.3026 prec: 0.1028


KeyboardInterrupt: 